In [78]:
import spotipy
import spotipy.util as util
import csv
import requests
import shutil
import urllib.request
from tqdm import tqdm
import billboard
import pandas as pd
import time
import numpy as np
from skimage import io
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

In [79]:
df = pd.read_pickle('./album_artwork_df.pkl').dropna()

In [80]:
df['genre_sum'] = df.apply(lambda x: x.rock+x.rap+x.country+x.latin+x.jazz+x.holiday,axis = 1)

In [81]:
class_names = ['rock','rap','country','latin','jazz','holiday']

In [82]:
def genre(row):
    if row.rock ==1:
        return(0)
    if row.rap ==1:
        return(1)
    if row.country ==1:
        return(2)
    if row.latin ==1:
        return(3)
    if row.jazz ==1:
        return(4)
    if row.holiday ==1:
        return(5)

In [116]:
def genre2(row):
    if row.rock ==1:
        return('rock')
    if row.rap ==1:
        return('rap')
    if row.country ==1:
        return('country')
    if row.latin ==1:
        return('latin')
    if row.jazz ==1:
        return('jazz')
    if row.holiday ==1:
        return('holiday')

In [85]:
genre(df.iloc[0])

0

In [83]:
df.head()

,artist,album,rock,rap,country,latin,jazz,holiday,image_url,genre_sum
0,The Rolling Stones,GRRR!,1.0,0.0,0.0,0.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b273196dd...,1.0
1,Crown The Empire,Retrograde,1.0,0.0,0.0,0.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b27318750...,1.0
2,Burn The Priest,Legion: XX,1.0,0.0,0.0,0.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b2735ba4b...,1.0
3,Miley Cyrus,Plastic Hearts,1.0,0.0,0.0,0.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b2738cffb...,1.0
4,Judas Priest,Firepower,1.0,0.0,0.0,0.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b2735c48f...,1.0


In [86]:
df['genre'] = df.apply(lambda x: genre(x),axis = 1)

In [122]:
df['genre2'] = df.apply(lambda x: genre2(x),axis = 1)

In [87]:
df['big_url']=df.image_url.apply(lambda x: x[0] )
df['small_url']=df.image_url.apply(lambda x: x[1] )

In [88]:
from io import BytesIO

def loadImage(URL):
    with urllib.request.urlopen(URL) as url:
        img = image.load_img(BytesIO(url.read()), target_size=(300, 300))

    return image.img_to_array(img)

In [89]:
tqdm.pandas()
df['image_vector'] = df.small_url.progress_apply(lambda x: loadImage(x))

100%|███████████████████████████████████████| 8412/8412 [19:15<00:00,  7.28it/s]


In [90]:
df.to_pickle('./album_artwork_vector.pkl')

In [149]:
# making it so there are no multi-genre albums
df_simple = df[df.genre_sum ==1]

In [92]:
len(df)

8412

In [93]:
len(df_simple)

8184

In [94]:
df_simple.sample(5)

,artist,album,rock,rap,country,latin,jazz,holiday,image_url,genre_sum,genre,big_url,small_url,image_vector
6558,Reggaeton Ninos,Ninos Vol. 2,0.0,0.0,0.0,1.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b273ee3eb...,1.0,3,https://i.scdn.co/image/ab67616d0000b273ee3ebe...,https://i.scdn.co/image/ab67616d00001e02ee3ebe...,"[[[254.0, 241.0, 4.0], [254.0, 241.0, 4.0], [2..."
2343,Kid Rock,Greatest Hits: You Never Saw Coming,1.0,0.0,0.0,0.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b27387cf8...,1.0,0,https://i.scdn.co/image/ab67616d0000b27387cf81...,https://i.scdn.co/image/ab67616d00001e0287cf81...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1345,Switchfoot,Oh! Gravity,1.0,0.0,0.0,0.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b27392b48...,1.0,0,https://i.scdn.co/image/ab67616d0000b27392b485...,https://i.scdn.co/image/ab67616d00001e0292b485...,"[[[13.0, 13.0, 13.0], [61.0, 61.0, 61.0], [60...."
7509,"Harry Connick, Jr.",Your Songs,0.0,0.0,0.0,0.0,1.0,0.0,(https://i.scdn.co/image/ab67616d0000b273aca69...,1.0,4,https://i.scdn.co/image/ab67616d0000b273aca693...,https://i.scdn.co/image/ab67616d00001e02aca693...,"[[[178.0, 210.0, 207.0], [209.0, 239.0, 237.0]..."
5912,Alegres de La Sierra,Duele El Amor,0.0,0.0,0.0,1.0,0.0,0.0,(https://i.scdn.co/image/ab67616d0000b27325906...,1.0,3,https://i.scdn.co/image/ab67616d0000b273259067...,https://i.scdn.co/image/ab67616d00001e02259067...,"[[[47.0, 27.0, 26.0], [56.0, 37.0, 33.0], [43...."


In [95]:
df_s = df_simple[['genre','image_vector']]

In [96]:
im_vec = df_simple['image_vector']
genres = df_simple['genre']

I gave up on trying to create a new dataset from vectors and am instead downloading images

In [151]:
import urllib.request

for index, row in df_simple.iterrows():
    urllib.request.urlretrieve(row.small_url, 'my_data/'+row.genre2+'/'+row.artist.replace('/','')+'_'+row.album.replace('/','')+'.jpg')